In [1]:
import pandas as pd
import numpy as np

In [28]:
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from joblib import load, dump
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, Normalizer, PolynomialFeatures
from Logger import RegressionLogger
import copy

In [61]:
X = load("objects/engineered_features.joblib")
model_log = load("objects/model_logging.joblib")
x_train, x_test, y_train, y_test, x_holdout, y_holdout = load("objects/holdout_total.joblib")

In [62]:
pd.DataFrame().from_dict(model_log)

,model,model_name,alpha,l1_ratio,rsquared,rmse,mae,train_rsquared,train_rmse,train_mae,...,max_depth,min_samples_leaf,objective,learning_rate,colsample_bytree,colsample_bynode,gamma,min_child_weight,importance_type,colsample_bylevel
0,"(ColumnTransformer(remainder='passthrough',\n ...",Elastic Net,0.1,0.5,0.559300,14.215243,9.888576,0.584740,14.147507,9.992870,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,0.1,0.5,0.562076,13.626806,9.479094,0.590908,14.080223,9.889907,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,NaN,NaN,0.571013,13.487057,9.450582,0.604599,13.842609,9.720690,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,NaN,NaN,0.565345,13.575853,9.430176,0.600432,13.915353,9.784322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,0.3,0.8,0.536707,14.015952,9.509088,0.634621,13.306718,9.332271,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"(FeatureUnion(transformer_list=[('numeric',\n ...",Random Forest,NaN,NaN,0.661840,11.974458,8.224307,0.908428,6.661632,4.355189,...,15.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"(FeatureUnion(transformer_list=[('numeric',\n ...",XGBRegressor,0.9,NaN,0.647749,12.221397,8.214601,0.978999,3.190186,2.344896,...,7.0,NaN,reg:squarederror,0.07,0.8,0.9,0.7,10.0,gain,NaN
7,"(FeatureUnion(transformer_list=[('numeric',\n ...",XGBRegressor,0.9,NaN,0.709581,11.073809,7.532104,0.984606,2.732257,2.017823,...,7.0,NaN,reg:squarederror,0.07,0.8,0.9,0.7,10.0,gain,1.0
8,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,0.3,0.8,0.592580,13.116117,9.050406,0.594010,14.031343,9.879924,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# _ = model_log.pop(7)
# _ = model_log.pop(7)
# _ = model_log.pop(7)

{'objective': 'reg:squarederror',
 'learning_rate': 0.1,
 'n_estimators': 100,
 'max_depth': 19,
 'colsample_bytree': 0.8,
 'colsample_bylevel': 0.9,
 'colsample_bynode': 0.9,
 'gamma': 0.7,
 'alpha': 0.8,
 'min_child_weight': 7,
 'importance_type': 'gain',
 'rsquared': 0.7095807306050848,
 'rmse': 11.073809458561588,
 'mae': 7.532104040014333,
 'train_rsquared': 0.9846057010352371,
 'train_rmse': 2.7322572842476402,
 'train_mae': 2.01782336314567,
 'model_name': 'XGBRegressor',
 'model': Pipeline(steps=[('preprocessing',
                  FeatureUnion(transformer_list=[('numeric',
                                                  Pipeline(steps=[('Impute',
                                                                   ColumnTransformer(transformers=[('Impute',
                                                                                                    KNNImputer(),
                                                                                                    <sklearn.c

In [63]:
elasticNet = copy.deepcopy(model_log[4]['model'])

In [47]:
params = {
    "model": elasticNet,
    "model_name": "Elastic Net",
    "alpha": 0.3,
    "l1_ratio": 0.8,
}
logger = RegressionLogger(params)

In [48]:
logger.train_holdout(elasticNet, x_train, x_test, y_train, y_test)

In [49]:
logger.save_log("Elastic Net with feature engineered features")

In [51]:
model_log.append(logger.record())

In [53]:
model_log[-1]["model"] = elasticNet

In [60]:
dump(model_log, "objects/model_logging.joblib")

['objects/model_logging.joblib']